In [1]:
import pandas as pd
import numpy as np
import os
import csv
from pathlib import Path
os.environ['cuda_visible_device'] = '0'

In [31]:
def get_data_RNA(df, gene_name, patient_ids):
    patient_dicts = []
    vals = []
    for col in content.columns:
        if ('TCGA' in col):
            p = col[:12]
            if p in patient_ids and '-01A' in col:  # only unhealthy samples (healthy will have >=10 instead of 01)
                curr_patient = {}
                curr_patient['id'] = p
                v = float(content.loc[content['gene_id']==gene_name, col].values[0])
                curr_patient['label'] = v
                patient_dicts.append(curr_patient)
                vals.append(v)
    vals = np.asarray(vals)
    mini = np.min(vals)
    maxi = np.max(vals)
    for p in patient_dicts:
        p['label'] = (p['label'] - mini) / (maxi - mini)
    return patient_dicts

    

In [29]:
label_type = 'RNA'
gene_name = 'MKI67|4288'
data_path = Path('../data/')
patient_id_file = Path('../data/patients_ids.txt')
patient_id = []
with open(patient_id_file, 'r') as f:
    patient_id = f.read().splitlines()
patient_id

['TCGA-A2-A25A',
 'TCGA-D8-A1XW',
 'TCGA-BH-A203',
 'TCGA-D8-A27M',
 'TCGA-AC-A2FO',
 'TCGA-E2-A107',
 'TCGA-AR-A0TP',
 'TCGA-E9-A1R6',
 'TCGA-A2-A04T',
 'TCGA-A2-A25D',
 'TCGA-BH-A18M',
 'TCGA-E2-A14P',
 'TCGA-C8-A27A',
 'TCGA-GM-A2DD',
 'TCGA-D8-A1JU',
 'TCGA-E9-A5FK',
 'TCGA-BH-A0HQ',
 'TCGA-D8-A1JL',
 'TCGA-A8-A08X']

In [4]:
if (label_type == 'RNA'):
    label_path = data_path/'EBPlusPlusAdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.tsv'
elif (label_type == 'MiRNA'):
    label_path = data_path/'pancanMiRs_EBadjOnProtocolPlatformWithoutRepsWithUnCorrectMiRs_08_04_16.csv'

In [5]:
content = pd.read_csv(label_path, sep='\t', header=0)

In [8]:
content.head()

,gene_id,TCGA-OR-A5J1-01A-11R-A29S-07,TCGA-OR-A5J2-01A-11R-A29S-07,TCGA-OR-A5J3-01A-11R-A29S-07,TCGA-OR-A5J5-01A-11R-A29S-07,TCGA-OR-A5J6-01A-31R-A29S-07,TCGA-OR-A5J7-01A-11R-A29S-07,TCGA-OR-A5J8-01A-11R-A29S-07,TCGA-OR-A5J9-01A-11R-A29S-07,TCGA-OR-A5JA-01A-11R-A29S-07,...,TCGA-CG-4449-01A-01R-1157-13,TCGA-CG-4462-01A-01R-1157-13,TCGA-CG-4465-01A-01R-1157-13,TCGA-CG-4466-01A-01R-1157-13,TCGA-CG-4469-01A-01R-1157-13,TCGA-CG-4472-01A-01R-1157-13,TCGA-CG-4474-01A-02R-1157-13,TCGA-CG-4475-01A-01R-1157-13,TCGA-CG-4476-01A-01R-1157-13,TCGA-CG-4477-01A-01R-1157-13
0,?|100130426,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,?|100133144,3.2661,2.6815,1.7301,0.0000,0.0000,1.1673,1.4422,0.0000,4.4556,...,4.358154,5.676995,5.219350,14.846708,20.115492,6.997533,18.311906,12.057112,18.628740,17.874417
2,?|100134869,3.9385,8.9948,6.5650,1.5492,4.4709,6.0529,2.2876,1.3599,5.0581,...,2.656360,3.342794,2.423442,5.055287,11.626054,13.654193,7.417109,11.585177,11.482418,14.919338
3,?|10357,149.1350,81.0777,86.4879,53.9117,66.9063,103.5060,94.9316,78.1955,69.2389,...,633.299781,294.018042,686.569179,563.573453,1039.307597,639.238135,742.479964,506.336449,712.452165,703.713324
4,?|10431,2034.1000,1304.9300,1054.6600,2350.8900,1257.9900,1866.4300,995.0270,1762.1200,1213.5300,...,1202.538277,644.002317,1181.884532,663.885074,647.530395,1297.152549,1152.909807,1375.495774,971.893874,1736.988111


In [32]:
v = content.loc[content['gene_id']==gene_name].values
v

array([['MKI67|4288', 108.069, 964.702, ..., 3967.76273146212,
        6036.78925159929, 9573.92802685253]], dtype=object)

In [33]:
values = get_data_RNA(content, gene_name, patient_id)

In [34]:
values


[{'id': 'TCGA-A2-A04T', 'label': 0.7430635852434544},
 {'id': 'TCGA-A2-A25A', 'label': 0.15077798506891874},
 {'id': 'TCGA-A2-A25D', 'label': 0.12364211816034353},
 {'id': 'TCGA-A8-A08X', 'label': 0.48219917726310885},
 {'id': 'TCGA-AC-A2FO', 'label': 0.03798256479364855},
 {'id': 'TCGA-AR-A0TP', 'label': 0.6563511931897211},
 {'id': 'TCGA-BH-A0HQ', 'label': 0.10145684067433453},
 {'id': 'TCGA-BH-A18M', 'label': 0.182844436641281},
 {'id': 'TCGA-BH-A203', 'label': 0.268023522772123},
 {'id': 'TCGA-C8-A27A', 'label': 1.0},
 {'id': 'TCGA-D8-A1JL', 'label': 0.3708903459152283},
 {'id': 'TCGA-D8-A1JU', 'label': 0.0},
 {'id': 'TCGA-D8-A1XW', 'label': 0.14634869612512527},
 {'id': 'TCGA-D8-A27M', 'label': 0.542950099423651},
 {'id': 'TCGA-E2-A107', 'label': 0.2839555480138922},
 {'id': 'TCGA-E2-A14P', 'label': 0.4134804773559074},
 {'id': 'TCGA-E9-A1R6', 'label': 0.15244897080223083},
 {'id': 'TCGA-E9-A5FK', 'label': 0.02614115969704793},
 {'id': 'TCGA-GM-A2DD', 'label': 0.14229126034451958}

In [35]:
header = ['id', 'label']
fname = f'../data/labels/label_{gene_name}.csv'
with open(fname, 'w', newline='') as myfile:
    writer = csv.DictWriter(myfile, fieldnames=header)
    writer.writeheader()
    writer.writerows(values)